# Lab Work #3
### Nedozhdii Oleksii FF-31mn

# Imports

In [1]:
import os
import random
import re
import string

import keras
import numpy as np
import splitfolders
import tensorflow as tf
import torch
from keras import layers
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from transformers import (
    BertForSequenceClassification,
    BertTokenizerFast,
    Trainer,
    TrainingArguments,
)

2024-12-28 11:08:53.266742: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/home/ned/.local/share/mamba/envs/data-scince/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# 3. Рекурентні нейронні мережі
Вирішіть задачу класифікації текстів (з якими ви працювали в лабораторній № 2) за допомогою рекурентної нейромережі двома способами:
а) навчить мережу і embedding шар з нуля (from scratch)
б) використовуючи pretrained word embeddings

## Load and prepare text data

In [2]:

multimodal_path = "resources/multimodal"
multimodal_path_splited = "resources/multimodal/splited"

In [3]:

# splitfolders.ratio(multimodal_path, output=multimodal_path_splited,
#                    seed=42, ratio=(.8, .1, .1))

In [4]:

raw_txt_train_ds = keras.utils.text_dataset_from_directory(
    os.path.join(multimodal_path_splited, "train"), batch_size=16, seed=42
)
raw_txt_val_ds = keras.utils.text_dataset_from_directory(
    os.path.join(multimodal_path_splited, "val"), batch_size=16, seed=42, shuffle=False
)
raw_txt_test_ds = keras.utils.text_dataset_from_directory(
    os.path.join(multimodal_path_splited, "test"), batch_size=16, seed=42, shuffle=False
)

Found 4663 files belonging to 6 classes.
Found 581 files belonging to 6 classes.
Found 587 files belonging to 6 classes.


In [5]:

txt_class_names = raw_txt_test_ds.class_names
txt_class_number = len(txt_class_names)
print(txt_class_names)
print(txt_class_number)

['damaged_infrastructure', 'damaged_nature', 'fires', 'flood', 'human_damage', 'non_damage']
6


In [6]:
def datatset_standardization(text, label):
    # print(raw_txt)
    text = tf.strings.lower(text)
    text = tf.strings.regex_replace(
        text, r"@\S+", " "
    )  # remove mentions
    text = tf.strings.regex_replace(
        text, r"https*\S+", " "
    )  # remove url
    text = tf.strings.regex_replace(
        text, r"#\S+", " "
    )  # remove hashtags
    text = tf.strings.regex_replace(
        text, r"\d", " "
    )  # remove all numbers
    # remove punctuations
    text = tf.strings.regex_replace(
        text, r"[%s]" % re.escape(string.punctuation), " "
    )
    text = tf.strings.regex_replace(
        text, r"\s{2,}", " "
    )  # remove extra spaces
    text = tf.strings.regex_replace(
        text, r"[^\x00-\x7F]+", ""
    )  # remove not ascii
    text = tf.strings.strip(text)
    return (text, label)

formated_txt_train_ds = raw_txt_train_ds.map(datatset_standardization)
formated_txt_test_ds = raw_txt_test_ds.map(datatset_standardization)
formated_txt_val_ds = raw_txt_val_ds.map(datatset_standardization)

formated_txt_train_ds = formated_txt_train_ds.unbatch().filter(lambda x, y: tf.strings.length(x) > 0)
formated_txt_test_ds = formated_txt_test_ds.unbatch().filter(lambda x, y: tf.strings.length(x) > 0)
formated_txt_val_ds = formated_txt_val_ds.unbatch().filter(lambda x, y: tf.strings.length(x) > 0)

formated_txt_train_ds = formated_txt_train_ds.batch(16)
formated_txt_test_ds = formated_txt_test_ds.batch(16)
formated_txt_val_ds = formated_txt_val_ds.batch(16)

In [7]:

for item in formated_txt_train_ds.unbatch():
    if item[0].numpy().decode("utf-8") == "":
        print("Spot emtpy line")

2024-12-28 11:09:15.971440: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [8]:

i = raw_txt_train_ds.unbatch().as_numpy_iterator()
print(next(i), end="\n\n")
print(next(i), end="\n\n")
print(next(i), end="\n\n")
del i

(b'Landslide below Meriema, between Kohima-Wokha road NH-61.\nTravellers are advised to take the route via secretariat road.\n#Nagaland #Wokha #Kohima #Meriema #Landslide\n\nFollow us and share News around you:\n\xf0\x9f\x91\x89 E-Mail us News around you @ wethenagas06@gmail.com\n\xf0\x9f\x91\x89 Direct Mail us or tag @wethenagas or hash tag #wethenagas\n\xf0\x9f\x91\x89 DM us for advertisements and business queries.', 1)

(b"Filled a swimming pool yesterday and will be paving over it because of #waterrestrictions it's actually a smart way of ensuring your pool is still around once we get through the #Drought. Your property value won't diminish but your water bill will. #ProgressThroughInnovation #VCT #VersatecCivils #VCTskips #CapeTown #WesternCape #FortheloveoftheHustle we also offer the drop, fill and collect option. We remove #GeneralWaste #Rubble #GardenRefuse #GarageCleanUps we provide Labour. #Affordable #Rates #ContactUsForAFreeQuote\xe2\x9e\xa1\xe2\x9e\xa1\xe2\x9e\xa1\xe2\x9e\

In [9]:

i = formated_txt_train_ds.as_numpy_iterator()
print(next(i), end="\n\n")
print(next(i), end="\n\n")
print(next(i), end="\n\n")
del i

(array([b'last years i have submitted over pieces of short concept jposters pajama time and copy to and all created on cellphones using and the phones being and all created by meme generator and image combiner i had to delete all but of these because of space someday i ll have my pc digital camera with all the bells and whistles poster quality printer and more cause amen it s gotta happen and much of that on these companies and such should be paying me i should be getting endorsements or something don t you think love your ricky hugsandkisses',
       b'miami florida',
       b'our sister island barbuda was devastated by hurricane irma restoration is underway but there is still much to be done to donate and give critical assistance to barbuda visit our website link in bio',
       b'y as nos invaden los colores', b'viennart k k hofburgtheater',
       b'had a sperstar a was an with soo', b'design by me',
       b'pour tre chic et tendance cet hiver  j ai craqu pour cet ensemble qui se 

In [10]:

max_features = 10000
sequence_length = 500

vectorize_layer = layers.TextVectorization(
    # standardize=datatset_standardization,
    max_tokens=max_features,
    output_mode="int",
    output_sequence_length=sequence_length,
)
train_text = formated_txt_train_ds.map(lambda x, y: x)
vectorize_layer.adapt(train_text)

2024-12-28 11:09:22.470667: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [11]:

def vectorize_text(text, label):
    text = tf.expand_dims(text, -1)
    return vectorize_layer(text), label

In [12]:

train_vec_txt_ds = formated_txt_train_ds.map(vectorize_text)
val_vec_txt_ds = formated_txt_val_ds.map(vectorize_text)
test_vec_txt_ds = formated_txt_test_ds.map(vectorize_text)

print(next(train_vec_txt_ds.as_numpy_iterator()))

(array([[  80,    4,  801, ...,    0,    0,    0],
       [  23,  230,  215, ...,    0,    0,    0],
       [2200, 2261,   15, ...,    0,    0,    0],
       ...,
       [   5,  906,  599, ...,    0,    0,    0],
       [ 372,  816, 5364, ...,    0,    0,    0],
       [   5, 1168,  360, ...,    0,    0,    0]]), array([5, 5, 1, 5, 4, 0, 1, 0, 0, 0, 1, 5, 0, 0, 0, 0], dtype=int32))


In [13]:

AUTOTUNE = tf.data.AUTOTUNE

train_txt_ds = train_vec_txt_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_txt_ds = val_vec_txt_ds.cache().prefetch(buffer_size=AUTOTUNE)
test_txt_ds = test_vec_txt_ds.cache().prefetch(buffer_size=AUTOTUNE)

## Model from scratch

In [14]:

emb_network = keras.Sequential()
emb_network.add(layers.Embedding(max_features, 16))
emb_network.add(layers.Dropout(0.2))
emb_network.add(layers.GlobalMaxPooling1D())
emb_network.add(layers.Dropout(0.2))
emb_network.add(layers.Dense(6, activation="softmax"))

emb_network.compile(
    loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"]
)
emb_network.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_max_pooling1d            │ ?                      │             0 │
│ (GlobalMaxPooling1D)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [15]:

history_emb_network = emb_network.fit(
    train_txt_ds, validation_data=val_txt_ds, epochs=25
)

Epoch 1/25
235/235 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.4082 - loss: 1.5944 - val_accuracy: 0.5087 - val_loss: 1.3996
Epoch 2/25
  1/235 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.5625 - loss: 1.1499

/home/ned/.local/share/mamba/envs/data-scince/lib/python3.12/site-packages/keras/src/trainers/epoch_iterator.py:151: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()
2024-12-28 11:10:03.065094: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


235/235 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.5195 - loss: 1.3776 - val_accuracy: 0.5563 - val_loss: 1.3144
Epoch 3/25
235/235 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.5595 - loss: 1.2771 - val_accuracy: 0.5931 - val_loss: 1.2211
Epoch 4/25
 38/235 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6140 - loss: 1.1315

2024-12-28 11:10:05.051088: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


235/235 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.5939 - loss: 1.1775 - val_accuracy: 0.6082 - val_loss: 1.1445
Epoch 5/25
235/235 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.6239 - loss: 1.0869 - val_accuracy: 0.6385 - val_loss: 1.0892
Epoch 6/25
235/235 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.6385 - loss: 1.0059 - val_accuracy: 0.6515 - val_loss: 1.0397
Epoch 7/25
235/235 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.6657 - loss: 0.9620 - val_accuracy: 0.6580 - val_loss: 1.0030
Epoch 8/25
 32/235 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.7142 - loss: 0.8541

2024-12-28 11:10:09.169292: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


235/235 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.6859 - loss: 0.9051 - val_accuracy: 0.6775 - val_loss: 0.9703
Epoch 9/25
235/235 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.7022 - loss: 0.8522 - val_accuracy: 0.6753 - val_loss: 0.9428
Epoch 10/25
235/235 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.7139 - loss: 0.8312 - val_accuracy: 0.6797 - val_loss: 0.9231
Epoch 11/25
235/235 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.7198 - loss: 0.7893 - val_accuracy: 0.6926 - val_loss: 0.9073
Epoch 12/25
235/235 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.7291 - loss: 0.7669 - val_accuracy: 0.6948 - val_loss: 0.8903
Epoch 13/25
235/235 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.7356 - loss: 0.7399 - val_accuracy: 0.7013 - val_loss: 0.8804
Epoch 14/25
235/235 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.7502 - loss: 0.7139 - val_accuracy: 0.7056 - val_loss: 0.8738
Epoch 15/25
235/235 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.7589 - loss: 0.6936 - val_accuracy: 0.7013 

2024-12-28 11:10:17.174767: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


235/235 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.7596 - loss: 0.6801 - val_accuracy: 0.7078 - val_loss: 0.8547
Epoch 17/25
235/235 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.7795 - loss: 0.6524 - val_accuracy: 0.7121 - val_loss: 0.8501
Epoch 18/25
235/235 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.7828 - loss: 0.6397 - val_accuracy: 0.7165 - val_loss: 0.8417
Epoch 19/25
235/235 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.7777 - loss: 0.6426 - val_accuracy: 0.7251 - val_loss: 0.8371
Epoch 20/25
235/235 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.7898 - loss: 0.6122 - val_accuracy: 0.7273 - val_loss: 0.8313
Epoch 21/25
235/235 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.7918 - loss: 0.5969 - val_accuracy: 0.7273 - val_loss: 0.8284
Epoch 22/25
235/235 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.8058 - loss: 0.6036 - val_accuracy: 0.7338 - val_loss: 0.8238
Epoch 23/25
235/235 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.8038 - loss: 0.5721 - val_accuracy: 0.7338

In [16]:

emb_loss, emb_accuracy = emb_network.evaluate(test_txt_ds)

print("Test loss:", emb_loss)
print("Test accuracy:", emb_accuracy)

30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6480 - loss: 1.0146
Test loss: 0.8289270401000977
Test accuracy: 0.7358490824699402


In [17]:

txt_true_labels = []

for _, label in test_txt_ds.unbatch():
    txt_true_labels.append(label.numpy())

In [18]:

y_predict_emb = emb_network.predict(test_txt_ds)
y_predict_emb = np.argmax(y_predict_emb, axis=1)
emb_network_score = accuracy_score(txt_true_labels, y_predict_emb)
emb_network_conf = confusion_matrix(txt_true_labels, y_predict_emb)
emb_network_report = classification_report(txt_true_labels, y_predict_emb)

print(f"Score: {emb_network_score}")

30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
Score: 0.7358490566037735


## Bert model

In [19]:

bert_tokenizer = BertTokenizerFast.from_pretrained("bert-base-uncased", max_length=512)
bert_model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased", num_labels=txt_class_number
)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [20]:
formated_txt_bert_ds = formated_txt_train_ds.unbatch()
text = formated_txt_bert_ds.map(lambda x, y: x)
text = list(np.fromiter(text.as_numpy_iterator(), dtype=((str, 512))))[:20]
txt_vec_bert_ds = bert_tokenizer(text, truncation=True, padding=True, return_tensors="pt")

In [21]:

bert_model.eval()
bert_output = bert_model(**txt_vec_bert_ds)

In [22]:

predictions = tf.nn.softmax(bert_output[0].detach(), axis=-1)

In [23]:

formated_txt_test_list = list(formated_txt_bert_ds.as_numpy_iterator())[:20]
print(formated_txt_test_list[0])
print([txt_class_names[i[1]] for i in formated_txt_test_list])

print(txt_class_names)
for i in range(20):
    print(f"\033[93mSource:\033[0m {text[i]}")
    true_class_code = formated_txt_test_list[i][1]
    pred_class_code = np.argmax(predictions[i])
    print(f"\033[32mtrue:\033[0m {txt_class_names[true_class_code]} | \033[31mpred:\033[0m {txt_class_names[pred_class_code]}")
    print()

(b'y as nos invaden los colores', 5)
['non_damage', 'damaged_infrastructure', 'non_damage', 'damaged_infrastructure', 'damaged_nature', 'damaged_nature', 'non_damage', 'non_damage', 'damaged_infrastructure', 'damaged_infrastructure', 'non_damage', 'damaged_infrastructure', 'non_damage', 'non_damage', 'non_damage', 'non_damage', 'non_damage', 'human_damage', 'damaged_nature', 'damaged_nature']
['damaged_infrastructure', 'damaged_nature', 'fires', 'flood', 'human_damage', 'non_damage']
Source: i m playing daily fantasy sports at fanduel com come join me and get to play terms apply 


twitch youtube
true: non_damage | pred: damaged_nature

Source: need a business card design for your brand or business professional cards only sided sided colddesignsbyicyy com
true: damaged_infrastructure | pred: damaged_nature

Source: guten morgen freunde der gesunden ernhrung 
heute mchte ich euch mal ein vorstellen es ist aber nicht nur einfach ein whey nein ist ist ein 
ich bin so froh endlich ein  gef